## Hands-on 01 - Python criando frames ficticios

### Importando bibliotecas

In [38]:
import pandas as pd
import numpy as np
from random import randint
from datetime import date
import datetime


### Mapeando, abrindo e salvando arquivos

In [108]:
base=pd.read_excel('../bases/vendas.xlsx')

### Criando frame de vendas

In [109]:
DATA=[]
PRODUTO=[]
SEGMENTO=[]
CPF=[]
VALOR=[]

for i in range(0,base.shape[0]):
    
    # DEFININDO ALEATORIAMENTE A DATA DE VENDA
    
    dia = randint(1,28)
    mes = randint(1,12)
    ano = 2018
    data = str(dia)+'-'+str(mes)+'-'+str(ano)

    
    # DEFININDO ALEATORIAMENTE O PRODUTO
    ale = randint(1,6)
    
    
    if ale == 1:
        p ='vida'
        v = randint(90,200)
    
    elif ale == 2:
        p = 'mulher'
        v = randint(70,150)
        
    elif ale == 3:
        p = 'senior'
        v = 50
    
    else:
        p = 'viva familia'
        v=20
    
    # DEFININDO ALEATORIAMENTE O SEGMENTO  
    
    seg = randint(1,5)
    
    if seg==1:
        s="personnalite"
    
    elif seg==2:
        s='uniclass'
    else:
        s='agencias'
        
    # DEFININDO ALEATORIAMENTE O CPF
    
    cpf=randint(333333333,444444444)
    
    DATA.append(data)
    PRODUTO.append(p)
    SEGMENTO.append(s)
    CPF.append(cpf)
    VALOR.append(v)
    
base['data']=DATA
base['data_venda']=pd.to_datetime(base.data,format='%d-%m-%Y')
base['produto']=PRODUTO
base['segmento']=SEGMENTO
base['cpf_semdac']=CPF
base['valor']=VALOR
base['anomes']=base.data_venda.apply(lambda x: x.strftime('%Y%m'))

base.drop('data',inplace=True,axis=1)
base.drop('cpf',inplace=True,axis=1)

base=base[['anomes','data_venda','cpf_semdac','segmento','apolice','produto','valor']]

#base.to_excel('../bases/vendas_final.xlsx')

In [110]:
base.head()

,anomes,data_venda,cpf_semdac,segmento,apolice,produto,valor
0,201809,2018-09-16,361769663,personnalite,111,viva familia,20
1,201806,2018-06-05,358968260,agencias,112,mulher,70
2,201805,2018-05-25,351490297,agencias,113,mulher,75
3,201807,2018-07-10,370619533,personnalite,114,mulher,126
4,201810,2018-10-25,353775735,agencias,115,viva familia,20


### Criando frame de potenciais

In [142]:
base_pot = pd.read_excel('../bases/vendas_final.xlsx')

In [143]:
POTENCIAL=[]

for i in range(0,base_pot.shape[0]):
    c=base_pot.apolice.loc[i]
    d= c % 5
    
    if d == 4:
        e = 'nao_potencial'
    else:
        e='potencial'
        
    POTENCIAL.append(e)

base_pot['potencial']=POTENCIAL

In [144]:
base_pot['produto']=base_pot.produto.shift(periods=1)

In [148]:
base_pot2=base_pot[['cpf_semdac','produto','potencial']]

In [150]:
base_pot3=base_pot2[(base_pot2.potencial == 'potencial') & (base_pot2.produto.notnull())]

In [154]:
base_pot3.to_excel("../bases/potencial_final.xlsx")

### Criando frame de cancelamento

In [50]:
base_cnc = pd.read_excel('../bases/vendas_final.xlsx')

In [51]:
CANC=[]

for i in range(0,base_cnc.shape[0]):
    c=base_cnc.apolice.loc[i]
    d= c % 3
    
    if d == 2:
        e = 'cancelado'
    else:
        e='ativo'
        
    CANC.append(e)

base_cnc['cancelado']=CANC

In [52]:
base_cnc.cancelado.value_counts()

ativo        860
cancelado    429
Name: cancelado, dtype: int64

In [53]:
base_cnc2=base_cnc[base_cnc.cancelado=='cancelado'].reset_index(drop=True)

In [54]:
X=[]
for i in range(0,base_cnc2.shape[0]):

    x = randint(1,360)    
    X.append(x)
    
base_cnc2['dias_canc'] = X

In [55]:
temp = base_cnc2['dias_canc'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))

In [56]:
base_cnc2['data_cancelamento']= base_cnc2.data_venda + temp

In [57]:
base_cnc3=base_cnc2[base_cnc2.data_cancelamento < '2019-01-01'][['apolice','produto','data_cancelamento']]

In [60]:
base_cnc3.to_excel('../bases/cancelamento_final.xlsx')